In [10]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping

#Loads the VGG16 model 
model = VGG16(weights = 'imagenet', include_top = False, input_shape = (244, 244, 3))

for layer in model.layers:
    layer.trainable = False
    
# Let's print our layers 
for (i,layer) in enumerate(model.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)
    
def Top(bottom_model, num_classes, D=256):
    head = bottom_model.output
    head = Flatten(name = "flatten")(head)
    head = Dense(D, activation = "relu")(head)
    head = Dropout(0.3)(head)
    head = Dense(num_classes, activation = "softmax")(head)
    return head

top = addTopModel(model, 4)
modelnew = Model(inputs=model.input, outputs=top)

print(modelnew.summary())

training = 'G:/MLOPS/transfer_learninig_vgg16/images/train'
testing = 'G:/MLOPS/transfer_learninig_vgg16/images/test'

train_data = ImageDataGenerator( rescale=1./255, rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
horizontal_flip=True, fill_mode='nearest')
 
test_data = ImageDataGenerator(rescale=1./255)
train_batchsize = 6
val_batchsize = 4
 
train_gen = train_datagen.flow_from_directory(training, target_size=(244, 244), batch_size=train_batchsize,
                                              class_mode='categorical')
 
test_gen = validation_datagen.flow_from_directory(testing, target_size=(244,244), batch_size=val_batchsize,
class_mode='categorical', shuffle=False)                   
checkpoint = ModelCheckpoint("actor.h5", monitor="val_loss", mode="min", save_best_only = True,
                             verbose=1)
earlystop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 3, verbose = 1, restore_best_weights = True)
callbacks = [earlystop, checkpoint]

modelnew.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.001),
              metrics = ['accuracy'])

train_img = 64
test_img = 16
epochs = 2
batch_size = 1

history = modelnew.fit_generator( train_gen, steps_per_epoch = train_img // batch_size, epochs = epochs,
callbacks = callbacks, validation_data = test_gen, validation_steps = test_img // batch_size)

modelnew.save("actor.h5")

0 InputLayer False
1 Conv2D False
2 Conv2D False
3 MaxPooling2D False
4 Conv2D False
5 Conv2D False
6 MaxPooling2D False
7 Conv2D False
8 Conv2D False
9 Conv2D False
10 MaxPooling2D False
11 Conv2D False
12 Conv2D False
13 Conv2D False
14 MaxPooling2D False
15 Conv2D False
16 Conv2D False
17 Conv2D False
18 MaxPooling2D False
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 244, 244, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 244, 244, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 244, 244, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 122, 122, 64)      0         
____________________________________________________________